In [66]:
import requests
import json
import time
from datetime import datetime, timedelta

In [106]:
r = requests.get('https://api.coingecko.com/api/v3/ping')

# Ping CoinGecko
print(r.ok)

True


In [101]:
def print_json(data: dict):
    data = json.dumps(data, indent=3)
    
    print(data)

In [109]:
def get_coingeko_stats(date: str, display_currency: str) -> dict:
    """Get cryptocurrency statistics from CoinGecko for a specified date:
    
    Parameters:
    - date: date in format dd-mm-yyyy
    - display_currency: usd, eur, gbp, czk, btc, eth, ... 
    """
    crypto_ids = ['bitcoin', 'ethereum', 'litecoin', 'bitcoin-cash']
    crypto_names = {
        'bitcoin': 'bitcoin',
        'ethereum': 'ethereum',
        'litecoin': 'litecoin',
        'bitcoin-cash': 'bitcoin_cash'
    }
    payload={'date':date}
    result = dict()
    
    for coin in crypto_ids:
        r = requests.get('https://api.coingecko.com/api/v3/coins/'+coin+'/history/', params=payload)
        data = json.loads(r.content)
                    
        market_data = data['market_data']
        
        coin_result = {
            'symbol': data['symbol'],
            'market_cap': market_data['market_cap'][display_currency],
            'current_price': market_data['current_price'][display_currency],
            'total_volume': market_data['total_volume'][display_currency]
        }
        
        result[crypto_names[coin]] = coin_result 

    return result

In [110]:
def get_glassnode_stats(date_start='01-01-2020', date_end='02-01-2020', display_currency='usd') -> dict:
    """Get on-chain cryptocurrency statistics from Glassnode for a specified date:
    
    Parameters:
    - date: date in format dd-mm-yyyy (gets converted to Unix timestamp)
    - display_currency: usd or native
    """
    API_KEY='1a6189c3-f990-45c7-970e-2d3b50bbc253'
    
    start_datetime = datetime.strptime(date_start, '%d-%m-%Y')
    end_datetime = datetime.strptime(date_end, '%d-%m-%Y')
    
    start_unixtime = int(time.mktime(start_datetime.timetuple()))
    end_unixtime = int(time.mktime(end_datetime.timetuple()))
    
    result = dict(bitcoin={}, ethereum={}, litecoin={}, bitcoin_cash={})
    
    crypto_ids = ['BTC', 'ETH', 'LTC', 'BCH']
    
    crypto_names = {
        'BTC': 'bitcoin',
        'ETH': 'ethereum',
        'LTC': 'litecoin',
        'BCH': 'bitcoin_cash'
    }
    
    request_endpoints = {
        'tx_fees':'fees/volume_mean',
        'active_adresses':'addresses/active_count',
        'tx_count':'transactions/count',
        'tx_mean_size':'transactions/transfers_volume_mean',
        'non_zero_adress_count':'addresses/non_zero_count'
    }
    
    for coin in crypto_ids:
        payload = {'a':coin,'api_key': API_KEY, 's':start_unixtime, 'u':end_unixtime, 'i':'24h', 'c':'usd'}
        for metric in request_endpoints:
            if metric == 'non_zero_adress_count':
                payload['a'] = 'BTC'
                r = requests.get('https://api.glassnode.com/v1/metrics/'+request_endpoints[metric], payload)
                data = json.loads(r.content)
                data = data[0]
                result[crypto_names['BTC']][metric] = data['v']
            else:
                r = requests.get('https://api.glassnode.com/v1/metrics/'+request_endpoints[metric], payload)
                data = json.loads(r.content)
                data = data[0]
                result[crypto_names[coin]][metric] = data['v']
                
    return result

In [111]:
def get_stats(date:str, display_currency='usd') -> dict:
    coingecko_ping = requests.get('https://api.coingecko.com/api/v3/ping')
    if coingecko_ping.ok:
        coingecko_data = get_coingeko_stats(date, 'usd')
    else:
        print('CoinGecko is down')
        return None
    
    datetime_begin = datetime.strptime(date, '%d-%m-%Y')
    datetime_end = datetime_begin + timedelta(days=1)
    
    date_begin = datetime_begin.strftime('%d-%m-%Y')
    date_end = datetime_end.strftime('%d-%m-%Y')
    
    glassnode_data = get_glassnode_stats(date_begin, date_end, 'usd')
    
    if glassnode_data == {}:
        print("Glassnode is down")
        return None
    
    crypto_names = ['bitcoin', 'ethereum', 'litecoin', 'bitcoin_cash']
    result = {
        'bitcoin':{},
        'ethereum':{},
        'litecoin':{},
        'bitcoin_cash':{}
    }
    for coin in crypto_names:
        coingecko_data_coin = coingecko_data[coin]
        glassnode_data_coin = glassnode_data[coin]
        result[coin] = {**coingecko_data_coin,**glassnode_data_coin}
    
    return result

In [112]:
print_json(get_stats('30-12-2017','usd'))

{
   "bitcoin": {
      "symbol": "btc",
      "market_cap": 228445816988.881,
      "current_price": 13620.3618741461,
      "total_volume": 3600481281.03768,
      "tx_fees": 32.32153712955091,
      "active_adresses": 1094021,
      "tx_count": 353824,
      "tx_mean_size": 95014.72717287869,
      "non_zero_adress_count": 26517339
   },
   "ethereum": {
      "symbol": "eth",
      "market_cap": 71238101432.8975,
      "current_price": 736.909191519636,
      "total_volume": 1306193221.81905,
      "tx_fees": 0.6787125022437158,
      "active_adresses": 543083,
      "tx_count": 1090191,
      "tx_mean_size": 13321.89684725972
   },
   "litecoin": {
      "symbol": "ltc",
      "market_cap": 12185489808.1574,
      "current_price": 223.400325158751,
      "total_volume": 569490495.299767,
      "tx_fees": 0.369016226768022,
      "active_adresses": 394056,
      "tx_count": 156079,
      "tx_mean_size": 55919.77483335565
   },
   "bitcoin_cash": {
      "symbol": "bch",
      "mark

In [95]:
print(get_coingeko_stats('30-12-2017','usd')['ethereum']) if r.ok else print("CoinGecko is down")

{'symbol': 'eth', 'market_cap': 71238101432.8975, 'current_price': 736.909191519636, 'total_volume': 1306193221.81905}


In [89]:
print(get_glassnode_stats())

{'bitcoin': {'tx_fees': 0.3672223907857314, 'active_adresses': 669834, 'tx_count': 293447, 'tx_mean_size': 17924.18497352796, 'non_zero_adress_count': 28219284}, 'ethereum': {'tx_fees': 0.08883370286043067, 'active_adresses': 192465, 'tx_count': 554215, 'tx_mean_size': 1545.089121284722}, 'litecoin': {'tx_fees': 0.02477757007505388, 'active_adresses': 108365, 'tx_count': 19651, 'tx_mean_size': 19574.10253698116}, 'bitcoin_cash': {'tx_fees': 0.004174442444671036, 'active_adresses': 58469, 'tx_count': 19716, 'tx_mean_size': 12325.760163306495}}
